In [2]:
import csv
import json
import pandas as pd
import polars as pl
from glob import glob
from tqdm import tqdm

In [3]:
def get_headers(file):
    with open(file, 'r', encoding='utf-8') as f:
        headers = f.readline().strip().split(',')
    return headers

In [4]:
files = glob('raw_data/IIEG/*.csv')
headers = list(map(get_headers, files))

data_headers = pd.DataFrame(headers)
data_headers.insert(0, 'file', files)

data_headers

,file,0,1,2,3,4,5,6,7,8,9,10
0,raw_data/IIEG/Altos_Sur_junio25.csv,"﻿""fecha""","""delito""","""x""","""y""","""colonia""","""municipio""","""clave_mun""","""hora""","""bien_afectado""","""zona_geografica""","""ID_municipio_unico"""
1,raw_data/IIEG/base_datos_incidencia_delictiva_...,﻿fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica,ID_municipio_unico
2,raw_data/IIEG/Lagunas_nov23.csv,"﻿""fecha""","""delito""","""x""","""y""","""colonia""","""municipio""","""clave_mun""","""hora""","""bien_afectado""","""zona_geografica""","""ID_municipio_unico"""
3,raw_data/IIEG/Costa_Sur_sep24.csv,"﻿""fecha""","""delito""","""x""","""y""","""colonia""","""municipio""","""clave_mun""","""hora""","""bien_afectado""","""zona_geografica""",None
4,raw_data/IIEG/Centro_agosto24.csv,"﻿""fecha""","""delito""","""x""","""y""","""colonia""","""municipio""","""clave_mun""","""hora""","""bien_afectado""","""zona_geografica""",None
5,raw_data/IIEG/Altos_Norte_mayo25.csv,"﻿""fecha""","""delito""","""x""","""y""","""colonia""","""municipio""","""clave_mun""","""hora""","""bien_afectado""","""zona_geografica""","""ID_municipio_unico"""
6,raw_data/IIEG/datos_incidencia_delicitva_sierr...,﻿fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica,None
7,raw_data/IIEG/Altos_Sur_junio24.csv,"﻿""fecha""","""delito""","""x""","""y""","""colonia""","""municipio""","""clave_mun""","""hora""","""bien_afectado""","""zona_geografica""",None
8,raw_data/IIEG/datos_incidencia_delictiva_sur_j...,"﻿""fecha""","""delito""","""x""","""y""","""colonia""","""municipio""","""clave_mun""","""hora""","""bien_afectado""","""zona_geografica""","""ID_municipio_unico"""
9,raw_data/IIEG/Sierra_Amula_ene24.csv,﻿fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica,None


In [5]:
def file_length(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)


def normalize_file(filename, dest_path, cols):
    chars = dict(zip('áéíóúÁÉÍÓÚñÑüÜ','aeiouAEIOUnNuU'))
    trtable = str.maketrans({**chars, ';': ':', '\n': ''})
    new_rows = []

    with open(filename, 'r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        for row in tqdm(reader, total=file_length(filename)):
            items = [cell.translate(trtable).lower().strip() for cell in row[:cols]]
            new_rows.append(items)

    with open(dest_path, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerows(new_rows)

In [7]:
[f for f in files if "25" in f]

['raw_data/IIEG/Altos_Sur_junio25.csv',
 'raw_data/IIEG/Altos_Norte_mayo25.csv',
 'raw_data/IIEG/datos_incidencia_delicitva_sierra_amula_jalisco_enero_2025.csv',
 'raw_data/IIEG/datos_incidencia_delictiva_sur_jalisco_feb_2025.csv',
 'raw_data/IIEG/Cienega_julio25.csv',
 'raw_data/IIEG/valles_abr25.csv',
 'raw_data/IIEG/base_datos_incidencia_delictiva_sureste_jalisco_mar2025.csv']

In [6]:
[f for f in files if "25" not in f]

['raw_data/IIEG/base_datos_incidencia_delictiva_region_lagunas_enero2017_nov2024.csv',
 'raw_data/IIEG/Lagunas_nov23.csv',
 'raw_data/IIEG/Costa_Sur_sep24.csv',
 'raw_data/IIEG/Centro_agosto24.csv',
 'raw_data/IIEG/Altos_Sur_junio24.csv',
 'raw_data/IIEG/Sierra_Amula_ene24.csv',
 'raw_data/IIEG/Sureste_mar24.csv',
 'raw_data/IIEG/Norte_diciembre23.csv',
 'raw_data/IIEG/Altos_Norte_mayo24.csv',
 'raw_data/IIEG/valles_abr24.csv',
 'raw_data/IIEG/base_datos_incidencia_delictiva_ragion_norte_2017_2024.csv',
 'raw_data/IIEG/Costa_Sierra_Occidental_oct24.csv',
 'raw_data/IIEG/Sur_feb24.csv',
 'raw_data/IIEG/Cienega_julio24.csv']

In [ ]:
for file in files:
    normalize_file(file, 'data/IIEG/' + file.split('/')[-1], 10)

100%|██████████| 10304/10304 [00:00<00:00, 85241.47it/s]


In [8]:
files = [f for f in glob('data/IIEG/*.csv') if "25" not in f]
df = pl.concat([pl.read_csv(f, infer_schema=False) for f in files])
df

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
str,str,str,str,str,str,str,str,str,str
"""12/01/2017""","""robo a casa habitacion""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""17:45""","""el patrimonio""","""interior"""
"""20/01/2017""","""lesiones dolosas""","""na""","""na""","""no disponible""","""tapalpa""","""86""","""05:30""","""la vida y la integridad corpor…","""interior"""
"""19/01/2017""","""robo a vehiculos particulares""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""05:30""","""el patrimonio""","""interior"""
"""13/01/2017""","""lesiones dolosas""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""05:30""","""la vida y la integridad corpor…","""interior"""
"""23/01/2017""","""violencia familiar""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""17:45""","""la familia""","""interior"""
…,…,…,…,…,…,…,…,…,…
"""2024-04-10""","""violencia familiar""","""na""","""na""","""no disponible""","""la barca""","""18""","""14:30""","""la familia""","""interior"""
"""2024-06-18""","""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""","""63""","""11:30""","""la familia""","""interior"""
"""2024-06-24""","""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""","""63""","""01:27""","""la familia""","""interior"""


In [9]:
df.filter(pl.col("x") == "na").count()

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
28595,28595,28595,28595,28595,28595,28595,28595,28595,28595


In [10]:
df.filter(pl.col("colonia") == "no disponible").count()

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
48511,48511,48511,48511,48511,48511,48511,48511,48511,48511


In [11]:
df.filter(~pl.col("hora").str.contains(r"^\d{1,2}:\d{2}$")).count()

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
169305,169305,169305,169305,169305,169305,169305,169305,169305,169305


In [12]:
def check_format(fecha):
    if '-' in fecha:
        y, m, d = map(int, fecha.split('-'))
        if y < 2016 or m > 12 or d > 31:
            return False
    elif '/' in fecha:
        d, m, y = map(int, fecha.split('/'))
        if y < 2016 or m > 12 or d > 31:
            return False
    return True


False in list(map(lambda x: check_format(x), df["fecha"].to_list()))

False

In [13]:
df.filter(~pl.col("fecha").str.contains(r"^\d{4}-\d{1,2}-\d{2}$|^\d{1,2}/\d{1,2}/\d{4}$"))

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
str,str,str,str,str,str,str,str,str,str


In [14]:
df = df.with_columns(
    pl.when(pl.col("fecha").str.contains(r"^\d{4}-\d{1,2}-\d{1,2}$"))
      .then(pl.col("fecha").str.strptime(pl.Date, "%Y-%m-%d", strict=False))
      .when(pl.col("fecha").str.contains(r"^\d{1,2}/\d{1,2}/\d{4}$"))
      .then(pl.col("fecha").str.strptime(pl.Date, "%d/%m/%Y", strict=False))
      .otherwise(None)
      .alias("fecha")
)

In [15]:
df

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
date,str,str,str,str,str,str,str,str,str
2017-01-12,"""robo a casa habitacion""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""17:45""","""el patrimonio""","""interior"""
2017-01-20,"""lesiones dolosas""","""na""","""na""","""no disponible""","""tapalpa""","""86""","""05:30""","""la vida y la integridad corpor…","""interior"""
2017-01-19,"""robo a vehiculos particulares""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""05:30""","""el patrimonio""","""interior"""
2017-01-13,"""lesiones dolosas""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""05:30""","""la vida y la integridad corpor…","""interior"""
2017-01-23,"""violencia familiar""","""na""","""na""","""no disponible""","""san martin hidalgo""","""77""","""17:45""","""la familia""","""interior"""
…,…,…,…,…,…,…,…,…,…
2024-04-10,"""violencia familiar""","""na""","""na""","""no disponible""","""la barca""","""18""","""14:30""","""la familia""","""interior"""
2024-06-18,"""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""","""63""","""11:30""","""la familia""","""interior"""
2024-06-24,"""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""","""63""","""01:27""","""la familia""","""interior"""


In [16]:
df.filter(pl.col("fecha").is_null())

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
date,str,str,str,str,str,str,str,str,str


In [17]:
df.filter(~df["clave_mun"].str.contains(r"^\d{1,3}$"))

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
date,str,str,str,str,str,str,str,str,str


In [18]:
df = df.with_columns(
    pl.col("clave_mun").cast(pl.Int32).alias("clave_mun")
)

df

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
date,str,str,str,str,str,i32,str,str,str
2017-01-12,"""robo a casa habitacion""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,"""17:45""","""el patrimonio""","""interior"""
2017-01-20,"""lesiones dolosas""","""na""","""na""","""no disponible""","""tapalpa""",86,"""05:30""","""la vida y la integridad corpor…","""interior"""
2017-01-19,"""robo a vehiculos particulares""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,"""05:30""","""el patrimonio""","""interior"""
2017-01-13,"""lesiones dolosas""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,"""05:30""","""la vida y la integridad corpor…","""interior"""
2017-01-23,"""violencia familiar""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,"""17:45""","""la familia""","""interior"""
…,…,…,…,…,…,…,…,…,…
2024-04-10,"""violencia familiar""","""na""","""na""","""no disponible""","""la barca""",18,"""14:30""","""la familia""","""interior"""
2024-06-18,"""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""",63,"""11:30""","""la familia""","""interior"""
2024-06-24,"""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""",63,"""01:27""","""la familia""","""interior"""


In [19]:
df = df.with_columns(
    pl.col("hora").str.strptime(pl.Time, "%H:%M", strict=False).alias("hora")
)

df

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
date,str,str,str,str,str,i32,time,str,str
2017-01-12,"""robo a casa habitacion""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,17:45:00,"""el patrimonio""","""interior"""
2017-01-20,"""lesiones dolosas""","""na""","""na""","""no disponible""","""tapalpa""",86,05:30:00,"""la vida y la integridad corpor…","""interior"""
2017-01-19,"""robo a vehiculos particulares""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,05:30:00,"""el patrimonio""","""interior"""
2017-01-13,"""lesiones dolosas""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,05:30:00,"""la vida y la integridad corpor…","""interior"""
2017-01-23,"""violencia familiar""","""na""","""na""","""no disponible""","""san martin hidalgo""",77,17:45:00,"""la familia""","""interior"""
…,…,…,…,…,…,…,…,…,…
2024-04-10,"""violencia familiar""","""na""","""na""","""no disponible""","""la barca""",18,14:30:00,"""la familia""","""interior"""
2024-06-18,"""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""",63,11:30:00,"""la familia""","""interior"""
2024-06-24,"""violencia familiar""","""na""","""na""","""no disponible""","""ocotlan""",63,01:27:00,"""la familia""","""interior"""


In [20]:
df = df.with_columns(
    pl.col("x").cast(pl.Float64, strict=False).alias("x"),
    pl.col("y").cast(pl.Float64, strict=False).alias("y")
)

df

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
date,str,f64,f64,str,str,i32,time,str,str
2017-01-12,"""robo a casa habitacion""",null,null,"""no disponible""","""san martin hidalgo""",77,17:45:00,"""el patrimonio""","""interior"""
2017-01-20,"""lesiones dolosas""",null,null,"""no disponible""","""tapalpa""",86,05:30:00,"""la vida y la integridad corpor…","""interior"""
2017-01-19,"""robo a vehiculos particulares""",null,null,"""no disponible""","""san martin hidalgo""",77,05:30:00,"""el patrimonio""","""interior"""
2017-01-13,"""lesiones dolosas""",null,null,"""no disponible""","""san martin hidalgo""",77,05:30:00,"""la vida y la integridad corpor…","""interior"""
2017-01-23,"""violencia familiar""",null,null,"""no disponible""","""san martin hidalgo""",77,17:45:00,"""la familia""","""interior"""
…,…,…,…,…,…,…,…,…,…
2024-04-10,"""violencia familiar""",null,null,"""no disponible""","""la barca""",18,14:30:00,"""la familia""","""interior"""
2024-06-18,"""violencia familiar""",null,null,"""no disponible""","""ocotlan""",63,11:30:00,"""la familia""","""interior"""
2024-06-24,"""violencia familiar""",null,null,"""no disponible""","""ocotlan""",63,01:27:00,"""la familia""","""interior"""


In [21]:
with open('data/regiones.json', 'r', encoding='utf-8') as f:
    regiones = json.load(f)

df = df.with_columns(
    pl.col("municipio")
    .replace_strict(regiones, default=None)
    .alias("region")
)

In [22]:
df

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica,region
date,str,f64,f64,str,str,i32,time,str,str,str
2017-01-12,"""robo a casa habitacion""",null,null,"""no disponible""","""san martin hidalgo""",77,17:45:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-20,"""lesiones dolosas""",null,null,"""no disponible""","""tapalpa""",86,05:30:00,"""la vida y la integridad corpor…","""interior""","""lagunas"""
2017-01-19,"""robo a vehiculos particulares""",null,null,"""no disponible""","""san martin hidalgo""",77,05:30:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-13,"""lesiones dolosas""",null,null,"""no disponible""","""san martin hidalgo""",77,05:30:00,"""la vida y la integridad corpor…","""interior""","""lagunas"""
2017-01-23,"""violencia familiar""",null,null,"""no disponible""","""san martin hidalgo""",77,17:45:00,"""la familia""","""interior""","""lagunas"""
…,…,…,…,…,…,…,…,…,…,…
2024-04-10,"""violencia familiar""",null,null,"""no disponible""","""la barca""",18,14:30:00,"""la familia""","""interior""","""cienega"""
2024-06-18,"""violencia familiar""",null,null,"""no disponible""","""ocotlan""",63,11:30:00,"""la familia""","""interior""","""cienega"""
2024-06-24,"""violencia familiar""",null,null,"""no disponible""","""ocotlan""",63,01:27:00,"""la familia""","""interior""","""cienega"""


---

In [23]:
df.write_parquet('data/iieg_data.parquet')

In [24]:
df = pl.read_parquet('data/iieg_data.parquet')
df

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica,region
date,str,f64,f64,str,str,i32,time,str,str,str
2017-01-12,"""robo a casa habitacion""",null,null,"""no disponible""","""san martin hidalgo""",77,17:45:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-20,"""lesiones dolosas""",null,null,"""no disponible""","""tapalpa""",86,05:30:00,"""la vida y la integridad corpor…","""interior""","""lagunas"""
2017-01-19,"""robo a vehiculos particulares""",null,null,"""no disponible""","""san martin hidalgo""",77,05:30:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-13,"""lesiones dolosas""",null,null,"""no disponible""","""san martin hidalgo""",77,05:30:00,"""la vida y la integridad corpor…","""interior""","""lagunas"""
2017-01-23,"""violencia familiar""",null,null,"""no disponible""","""san martin hidalgo""",77,17:45:00,"""la familia""","""interior""","""lagunas"""
…,…,…,…,…,…,…,…,…,…,…
2024-04-10,"""violencia familiar""",null,null,"""no disponible""","""la barca""",18,14:30:00,"""la familia""","""interior""","""cienega"""
2024-06-18,"""violencia familiar""",null,null,"""no disponible""","""ocotlan""",63,11:30:00,"""la familia""","""interior""","""cienega"""
2024-06-24,"""violencia familiar""",null,null,"""no disponible""","""ocotlan""",63,01:27:00,"""la familia""","""interior""","""cienega"""


In [25]:
df.filter(pl.col("colonia") == "no disponible").filter(~pl.col("x").is_null())

fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica,region
date,str,f64,f64,str,str,i32,time,str,str,str
2017-01-05,"""robo a carga pesada""",-103.576098,20.435153,"""no disponible""","""acatlan de juarez""",2,17:45:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-04,"""robo a casa habitacion""",-103.957391,20.37223,"""no disponible""","""san martin hidalgo""",77,05:30:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-03,"""robo de motocicleta""",-103.391568,20.134509,"""no disponible""","""teocuitatlan de corona""",92,17:45:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-12,"""robo a vehiculos particulares""",-103.591631,20.336282,"""no disponible""","""acatlan de juarez""",2,05:30:00,"""el patrimonio""","""interior""","""lagunas"""
2017-01-11,"""robo a vehiculos particulares""",-103.293515,20.107573,"""no disponible""","""teocuitatlan de corona""",92,17:45:00,"""el patrimonio""","""interior""","""lagunas"""
…,…,…,…,…,…,…,…,…,…,…
2024-06-24,"""violencia familiar""",-102.600779,20.464529,"""no disponible""","""atotonilco el alto""",13,14:40:00,"""la familia""","""interior""","""cienega"""
2024-06-26,"""violencia familiar""",-102.963273,20.32286,"""no disponible""","""poncitlan""",66,15:40:00,"""la familia""","""interior""","""cienega"""
2024-07-03,"""violencia familiar""",-102.357734,20.525367,"""no disponible""","""ayotlan""",16,19:00:00,"""la familia""","""interior""","""cienega"""
